## Children Household Count Per County File Processing

In this file, we will be making a dataframe that includes the number of children in all of the households in each county of the US.

### Import statements

In [1]:
import pandas as pd
import geopandas as gpd
import os

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Setting ```DATA_DIR```
In order to read in files from this repository, we must set ```DATA_DIR``` to be the data folder within this repository. This requires ```os.getcwd()``` to return the path to the processing notebook of this repository, so ```xxx/codeplus-celine-dcc-package/procesing```, where ```xxx``` is the path to where you cloned this repository. If it is not, use ```os.chdir(path)``` to change the current working directory to ```xxx/codeplus-celine-dcc-package/procesing``` before getting the current working directory in ```DATA_DIR = os.getcwd()```, where ```path``` is ```xxx/codeplus-celine-dcc-package/procesing```.

In [30]:
DATA_DIR = os.getcwd()
DATA_DIR = DATA_DIR.replace('processing', 'data')
DATA_DIR

'/hpc/home/at341/ondemand/codeplus-celine-dcc-package/data'

### Importing county level shape file

This shapefile of US counties was taken from  the United States Census Bureau's Cartographic Boundary Files (available [here](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html)). It file provides the polygon geometries of each county in the US; this will allow us to make visualizations of the US with breakdowns by county. In this case, we are making this processing file to allow us make a map of US with children count by county.

In [21]:
df_counties = gpd.read_file(DATA_DIR + '/source_files/county_shapefiles/counties.shp')
df_counties['county_fips'] = df_counties['STATEFP'] + df_counties['COUNTYFP']
df_counties = df_counties[((df_counties['STATEFP'] != '02') & (df_counties['STATEFP'] != '15') & 
                           (df_counties['STATEFP'] != '72') & (df_counties['STATEFP'] != '78') &
                           (df_counties['STATEFP'] != '60') & (df_counties['STATEFP'] != '66') &
                          (df_counties['STATEFP'] != '69'))]
df_counties.head(n=3)

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry,county_fips
0,20,161,00485044,0500000US20161,20161,Riley,Riley County,KS,Kansas,06,1579077672,32047392,"POLYGON ((-96.96095 39.28670, -96.96106 39.288...",20161
1,19,159,00465268,0500000US19159,19159,Ringgold,Ringgold County,IA,Iowa,06,1386932347,8723135,"POLYGON ((-94.47167 40.81255, -94.47166 40.819...",19159
2,30,009,01720111,0500000US30009,30009,Carbon,Carbon County,MT,Montana,06,5303728455,35213028,"POLYGON ((-109.79867 45.16734, -109.68779 45.1...",30009


### Reading InfoUSA data

This pre-processed InfoUSA household file is all the merged InfoUSA files combined in processing notebok **01_merging_files**. It contains the children counts of households as well as county FIPS of each household. This file also includes the transformed latitude and longitude coordinates.

In this dataset, the ```county_fips``` column represents each individual county; this is the column we will be grouping the children count by. After reading in the data, we filter for only the columns we will need for our visualizations.

In [22]:
df_hh = pd.read_parquet(DATA_DIR + '/infousa_merged.parquet')
df_hh.head()

,zip,county_fips,state,child_num,has_child,age_code,lat_h_4326,lon_h_4326,lat_h_3857,lon_h_3857
0,84606,37041,NH,12,0,H,47.536477,-123.299151,-1.372560e+07,6.030085e+06
1,66723,23407,OR,3,0,A,43.121187,-123.473248,-1.374498e+07,5.330436e+06
2,59965,50536,NV,7,1,I,38.462998,-111.014933,-1.235813e+07,4.645040e+06
3,38676,38340,SD,3,1,B,48.433631,-80.411200,-8.951334e+06,6.179301e+06
4,75640,24383,OR,15,1,J,42.203405,-73.802723,-8.215682e+06,5.191497e+06


In [23]:
df_hh = df_hh[['county_fips', 'state', 'child_num', 'has_child', 'age_code']]
df_hh.head()

,county_fips,state,child_num,has_child,age_code
0,37041,NH,12,0,H
1,23407,OR,3,0,A
2,50536,NV,7,1,I
3,38340,SD,3,1,B
4,24383,OR,15,1,J


### Using ```.groupby()``` to find the number of children per county

Here, we are taking the dataframe from above and calculating the number of children per county. The ```.groupby()``` method groups the ```df_hh``` by ```county_fips```, and the ```.sum()``` method sums the values in ```child_num``` in each ```county_fips``` group. The resulting dataframe, ```df_child_count```, has the number of children in each county.

In [24]:
df_child_count = df_hh.groupby('county_fips')['child_num'].sum().reset_index()
df_child_count.head()

,county_fips,child_num
0,00000,191
1,00001,148
2,00002,198
3,00003,180
4,00004,155


### Using ```.merge()``` to get processed dataframe

In order to map this dataframe visually, we must have a geometry for each county. To do this, we merge ```df_child_count``` from above with ```df_counties``` ```on``` the ```county_fips``` column. Afterwards, we will select only the columns we want to keep in this dataframe and rename column names for standardization purposes before exporting this as a parquet file to be used in future visualizations.

In [25]:
df = df_counties.merge(df_child_count, on = ['county_fips'], how = 'left')
df.head(n=3)

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry,county_fips,child_num
0,20,161,00485044,0500000US20161,20161,Riley,Riley County,KS,Kansas,06,1579077672,32047392,"POLYGON ((-96.96095 39.28670, -96.96106 39.288...",20161,111.0
1,19,159,00465268,0500000US19159,19159,Ringgold,Ringgold County,IA,Iowa,06,1386932347,8723135,"POLYGON ((-94.47167 40.81255, -94.47166 40.819...",19159,239.0
2,30,009,01720111,0500000US30009,30009,Carbon,Carbon County,MT,Montana,06,5303728455,35213028,"POLYGON ((-109.79867 45.16734, -109.68779 45.1...",30009,138.0


In [27]:
df = df[['STATEFP', 'NAME', 'county_fips','child_num', 'geometry']]
df.head()

,STATEFP,NAME,county_fips,child_num,geometry
0,20,Riley,20161,111.0,"POLYGON ((-96.96095 39.28670, -96.96106 39.288..."
1,19,Ringgold,19159,239.0,"POLYGON ((-94.47167 40.81255, -94.47166 40.819..."
2,30,Carbon,30009,138.0,"POLYGON ((-109.79867 45.16734, -109.68779 45.1..."
3,16,Bear Lake,16007,112.0,"POLYGON ((-111.63452 42.57034, -111.63010 42.5..."
4,55,Buffalo,55011,143.0,"POLYGON ((-92.08384 44.41200, -92.08310 44.414..."


In [28]:
df.rename(columns = {'STATEFP': 'state', 'NAME': 'county'}, inplace = True)

In [29]:
df.to_file(DATA_DIR + '/children_count_by_county.shp')

/tmp/ipykernel_1405578/3883968385.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df.to_file(DATA_DIR + '/children_count_by_county.shp')
